학습 데이터를 만든다.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# xData => [공부 시간, 과외 시간]
temp = [[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,9]]
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# yData => [합격 여부] => 실제값
temp =[0, 0, 0, 1, 1, 1, 1]
print(temp)
print(type(temp))

yData = np.array(temp)  #1행 7열
print(yData)
print(type(yData))
#reshape() 함수는 데이터는 그댈로 유지한 채 배열의 형태(차원, 구조)를 변경한다. 1행 7열 넘파이 배열 => 7향 1열 넘파이 배열
yData = np.array(temp).reshape(7, 1)
print(yData)
print(type(yData))


[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장함 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해 대입되는 넘파이 배열의 차원을 지정해줘야 한다.
# [None, 2] => placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관이 없고 열의 개수는 무조건 2개이다.
X = tf.placeholder(dtype = tf.float32, shape = [None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype = tf.float32, shape = [None, 1]) # yData가 대입될 placeholder

기울기(가충치) a와 y절편(바이어스) b를 임의로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2,1]), dtype = tf.float32) # 난수를 2행 1열로 만든다.
b = tf.Variable(tf.random_uniform([1]), dtype = tf.float32) # 난수를 2행 1열로 만든다

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#print('a:\n{}'.format(sess.run(a)))
#print('b:\n{}'.format(sess.run(b)))

print('a1: {}, a2: {}, b:{}'.format(sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a1: [0.271222], a2: [0.88035107], b:[0.92466784]


a 기울기                
[[0.09128368]
 [0.48712146]]    
 
 
b y절판    
[0.4934343]     
   
   
(x1 + a1) + (x2 * a2)     =>      2 * 0.09128366 + 3 * 0.48712146


시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# sigmoid(): tensorflow에서 시그모이드 방정식을 계산하는 함수
# matmul(): tensorflow에서 행렬의 곱셈을 계산하는 함수
y = tf.sigmoid(tf.matmul(X, a) + b)  # 시그모이드 방정식
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 강사 하강

sigmoid() 함수의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 0을 리턴시킨다

In [7]:
sess = tf.Session()
# cast(캐스팅할 데이터, dtype = 캐스팅될 데이터 타입): tensorflow에서 형변환을 하는 함수
# predicted = tf.cast(tf.constant([1.9, 2.1]), dtype = tf.int32)
# print(sess.run(predicted)) # [1 2]

# cast() 함수의 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅 된다.


In [8]:
predicted = tf.cast(0.4 >= 0.5, dtype = tf.float32)
print(sess.run(predicted))
predicted = tf.cast(0.5 >= 0.5, dtype = tf.float32)
print(sess.run(predicted))

0.0
1.0


sigmoid() 함수를 실행한 예상값을 계산한다.

In [9]:
predicted = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid() 함수를 실행한 결과(y)를 0 또는 1로 변환한다.
# 예측값이 실제값과 일치하는 정도(정확도)를 계산한다.
# equal(): tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 함수
# equal() 함수로 예측값(predicted) 실제값(Y)이 같은가 비교한 후 True 또는 False 결과를 1 또는 0으로 캐스팅하고 실행된
# 전체 결과의 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

학습을 시킨다.

In [32]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001):
    # sess.run([기울기, y절편, 오차함수, 경사하강], feed_dict = {X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    # 연산은 하긴 하는데 사용하지 않을 실행 결과를 기억하는 변수는 '_'로 사용한다.
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict = {X: xData, Y: yData})
    if i % 300 == 0:
        # Epoch: 학습 횟수, a1: 공부 시간 기울기, a2: 과외 시간 기울기, b: y절편, loss: 오차함수, gradient: 경사하강
        print("Epoch: {0:4d}, a1: {1:9.5f}, a2: {2:9.5f}, b: {3:9.5f}, loss: {4:9.5f}".format(i, a_[0][0], a_[1][0], b_[0], loss_))
        
        
# ============================
print('=' * 80)
# 테스트 데이터를 만든다.
# new_x = np.array([6, 5]) # 테스트 데이터가 1차원 넘파이 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x)       # [6 5]   => 1차원
# print(new_x.shape) # (2,)    => 1차원
# print(new_x.ndim)  # 1       => 1차원
# reshape() 함수로 테스트 데이터를 1행 2열인 2차원 넘파이 배열로 변환해야 행렬의 곱셈 연산이 제대로 실행된다.
#new_x = np.array([6, 5]).reshape(1, 2) # 2 차원으로 만들어야 한다
# print(new_x)       # [[6 5]] => 2차원
# print(new_x.shape) # (1, 2)  => 2차원
# print(new_x.ndim)  # 2       => 2차원

# sigmoid() 함수의 연산 결과에 테스트 데이터를 대입해서 연산한다.
# result , new_y = sess.run([predicted, y], feed_dict = {X: new_x})
# print("공부 시간: {}, 과외 시간: {}".format(new_x[0][0], new_x[0][1]))
# print("시험 예측 결과: {0:s}, 합격 확률: {1:6.2f}%".format('합격' if result[0][0] == 1 else '불합격', new_y[0][0] * 100))  # 합격 확률 51.1%

# 공부는 1시간도 안하고 과외 수업 0 ~ 10 시간을 받알을 때 합격 여부와 합격 확률
for i in range(11):
    new_x = np.array([0, i]).reshape(1, 2)
    result , new_y = sess.run([predicted, y], feed_dict = {X: new_x})
    print("공부 시간: {}, 과외 시간: {}".format(new_x[0][0], new_x[0][1]))
    print("시험 예측 결과: {0:s}, 합격 확률: {1:6.2f}%".format('합격' if result[0][0] == 1 else '불합격', new_y[0][0] * 100))  # 합격 확률 51.1%

print('=' * 80)

for i in range(11):
    new_x = np.array([i, 0]).reshape(1, 2)
    result , new_y = sess.run([predicted, y], feed_dict = {X: new_x})
    print("공부 시간: {}, 과외 시간: {}".format(new_x[0][0], new_x[0][1]))
    print("시험 예측 결과: {0:s}, 합격 확률: {1:6.2f}%".format('합격' if result[0][0] == 1 else '불합격', new_y[0][0] * 100))  # 합격 확률 51.1%


Epoch:    0, a1:   0.25371, a2:   0.56301, b:   0.61976, loss:   2.02138
Epoch:  300, a1:   0.79562, a2:  -0.53236, b:  -2.39672, loss:   0.26887
Epoch:  600, a1:   0.80760, a2:  -0.27536, b:  -3.87906, loss:   0.19209
Epoch:  900, a1:   0.72342, a2:   0.04492, b:  -4.94761, loss:   0.15009
Epoch: 1200, a1:   0.62329, a2:   0.34693, b:  -5.79191, loss:   0.12284
Epoch: 1500, a1:   0.52668, a2:   0.61724, b:  -6.49167, loss:   0.10371
Epoch: 1800, a1:   0.43878, a2:   0.85625, b:  -7.08978, loss:   0.08959
Epoch: 2100, a1:   0.36041, a2:   1.06762, b:  -7.61229, loss:   0.07878
Epoch: 2400, a1:   0.29090, a2:   1.25541, b:  -8.07630, loss:   0.07025
Epoch: 2700, a1:   0.22922, a2:   1.42332, b:  -8.49369, loss:   0.06337
Epoch: 3000, a1:   0.17427, a2:   1.57448, b:  -8.87303, loss:   0.05770
공부 시간: 0, 과외 시간: 0
시험 예측 결과: 불합격, 합격 확률:   0.01%
공부 시간: 0, 과외 시간: 1
시험 예측 결과: 불합격, 합격 확률:   0.07%
공부 시간: 0, 과외 시간: 2
시험 예측 결과: 불합격, 합격 확률:   0.33%
공부 시간: 0, 과외 시간: 3
시험 예측 결과: 불합격, 합격 확률:   1.55%
공

Epoch: 3000, a1:   0.17226, a2:   1.58036, b:  -8.88866로 정확하게 예측하기